In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import datetime

In [2]:
def preprocess_data(data):
    data_train = pd.DataFrame(data.Close[0: int(len(data)*0.80)])
    data_test = pd.DataFrame(data.Close[int(len(data)*0.80): len(data)])

    scaler = MinMaxScaler(feature_range=(0, 1))
    past_100_days = data_train.tail(100)
    data_test = pd.concat([past_100_days, data_test], ignore_index=True)
    data_test_scale = scaler.fit_transform(data_test)

    return data_test_scale, scaler

In [3]:
def make_predictions(model, data_test_scale, scaler):
    x, y = [], []
    for i in range(100, data_test_scale.shape[0]):
        x.append(data_test_scale[i-100:i])
        y.append(data_test_scale[i, 0])
    x, y = np.array(x), np.array(y)
    predict = model.predict(x)
    predict = scaler.inverse_transform(predict)
    y = scaler.inverse_transform(y.reshape(-1, 1))
    return predict, y.flatten()

In [4]:

def predict_future_prices_with_dates_15(model, scaler, latest_data, num_future_days=15):
    future_dates = [datetime.date.today() + datetime.timedelta(days=i) for i in range(1, num_future_days + 1)]

    # Scale the latest data
    latest_data_scaled = scaler.transform(latest_data.reshape(-1, 1))

    # Prepare input data for prediction
    x_input = np.array([latest_data_scaled[-100:].flatten()])

    # Reshape the input data for LSTM layer
    x_input = np.reshape(x_input, (x_input.shape[0], x_input.shape[1], 1))

    # Make predictions for the next 'num_future_days' days
    future_predictions = []
    for _ in range(num_future_days):
        next_day_prediction = model.predict(x_input)[0, 0]
        future_predictions.append(next_day_prediction)
        x_input = np.roll(x_input, shift=-1, axis=1)
        x_input[0, -1, 0] = next_day_prediction

    # Inverse transform the predictions
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    return future_dates, future_predictions.flatten()

In [5]:

def predict_future_prices_with_dates_30(model, scaler, latest_data, num_future_days=30):
    future_dates = [datetime.date.today() + datetime.timedelta(days=i) for i in range(1, num_future_days + 1)]

    # Scale the latest data
    latest_data_scaled = scaler.transform(latest_data.reshape(-1, 1))

    # Prepare input data for prediction
    x_input = np.array([latest_data_scaled[-100:].flatten()])

    # Reshape the input data for LSTM layer
    x_input = np.reshape(x_input, (x_input.shape[0], x_input.shape[1], 1))

    # Make predictions for the next 'num_future_days' days
    future_predictions = []
    for _ in range(num_future_days):
        next_day_prediction = model.predict(x_input)[0, 0]
        future_predictions.append(next_day_prediction)
        x_input = np.roll(x_input, shift=-1, axis=1)
        x_input[0, -1, 0] = next_day_prediction

    # Inverse transform the predictions
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    return future_dates, future_predictions.flatten()

In [6]:
def predict_for_multiple_stocks_15(model_path, stock_list, start_date, end_date):
    results = []

    # Load the pre-trained model
    model = load_model(model_path)

    for stock_symbol in stock_list:
       data = yf.download(stock_symbol, start_date, end_date)
       data_test_scale, scaler = preprocess_data(data)
       ma_50_days = data.Close.rolling(50).mean()
       ma_100_days = data.Close.rolling(100).mean()
       ma_200_days = data.Close.rolling(200).mean()
    # fig_ma = plot_moving_averages(data, ma_50_days, ma_100_days, ma_200_days)
       predict, y = make_predictions(model, data_test_scale, scaler)
    # fig_pred = plot_predictions(predict, y)
    # Get only the last 30 rows of the data
    # data_last_30_rows = data.tail(20).to_html()
    # Predict future prices for the next 15 days with date labels
       latest_data = data.Close.values[-100:]
       future_dates, future_predictions = predict_future_prices_with_dates_15(model, scaler, latest_data, num_future_days=15)
 
       results.append([stock_symbol, future_predictions.tolist()])

    return results

In [7]:
def predict_for_multiple_stocks_30(model_path, stock_list, start_date, end_date):
    results = []

    # Load the pre-trained model
    model = load_model(model_path)

    for stock_symbol in stock_list:
       data = yf.download(stock_symbol, start_date, end_date)
       data_test_scale, scaler = preprocess_data(data)
       ma_50_days = data.Close.rolling(50).mean()
       ma_100_days = data.Close.rolling(100).mean()
       ma_200_days = data.Close.rolling(200).mean()
    # fig_ma = plot_moving_averages(data, ma_50_days, ma_100_days, ma_200_days)
       predict, y = make_predictions(model, data_test_scale, scaler)
    # fig_pred = plot_predictions(predict, y)
    # Get only the last 30 rows of the data
    # data_last_30_rows = data.tail(20).to_html()
    # Predict future prices for the next 15 days with date labels
       latest_data = data.Close.values[-100:]
       future_dates, future_predictions = predict_future_prices_with_dates_30(model, scaler, latest_data, num_future_days=30)
 
       results.append([stock_symbol, future_predictions.tolist()])

    return results

In [8]:
model_path = 'Stock_Model.keras'
stock_list = ['ADANIENT.NS', 'ASIANPAINT.NS', 'BAJAJFINSV.NS', 'BHARTIARTL.NS', 'BOSCHLTD.NS', 'BRFL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DLF.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GAIL.NS', 'GRASIM.NS', 'HDFCBANK.NS', 'HINDALCO.NS', 'HCLTECH.NS','ICICIBANK.NS', 'INFY.NS', 'ITC.NS', 'JINDALSTEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'MARUTI.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TATACHEM.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'UPL.NS', 'WIPRO.NS']
start_date = '2023-01-01'
end_date = '2023-11-29'

predictions_15 = predict_for_multiple_stocks_15(model_path,stock_list, start_date, end_date)
predictions_30 = predict_for_multiple_stocks_30(model_path,stock_list, start_date, end_date)


[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - 0s 40ms/step


C:\Users\AKHIL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 35ms/step
[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - 0s 36ms/step


C:\Users\AKHIL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 52ms/step
[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - 0s 38ms/step


...

1/1 [==============================] - 0s 37ms/step
[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - ETA: 0s

C:\Users\AKHIL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 51ms/step
[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - 0s 45ms/step


C:\Users\AKHIL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 35ms/step


In [9]:
predictions_15

[['ADANIENT.NS',
  [2191.823486328125,
   2199.892578125,
   2208.26953125,
   2214.522216796875,
   2217.90771484375,
   2218.7626953125,
   2217.788330078125,
   2215.65966796875,
   2212.922119140625,
   2209.94970703125,
   2206.96484375,
   2204.082275390625,
   2201.3486328125,
   2198.7802734375,
   2196.368408203125]],
 ['ASIANPAINT.NS',
  [3109.671142578125,
   3113.764404296875,
   3117.280517578125,
   3120.18896484375,
   3122.4951171875,
   3124.2685546875,
   3125.612060546875,
   3126.63330078125,
   3127.426513671875,
   3128.0673828125,
   3128.613525390625,
   3129.102294921875,
   3129.55712890625,
   3129.990966796875,
   3130.407958984375]],
 ['BAJAJFINSV.NS',
  [1598.54150390625,
   1599.5108642578125,
   1597.946533203125,
   1594.17626953125,
   1588.8853759765625,
   1582.72314453125,
   1576.2005615234375,
   1569.6663818359375,
   1563.3565673828125,
   1557.44482421875,
   1552.0087890625,
   1547.0614013671875,
   1542.581298828125,
   1538.5286865234375,
 

In [10]:
predictions_30

[['ADANIENT.NS',
  [2191.823486328125,
   2199.892578125,
   2208.26953125,
   2214.522216796875,
   2217.90771484375,
   2218.7626953125,
   2217.788330078125,
   2215.65966796875,
   2212.922119140625,
   2209.94970703125,
   2206.96484375,
   2204.082275390625,
   2201.3486328125,
   2198.7802734375,
   2196.368408203125,
   2194.095947265625,
   2191.947509765625,
   2189.91064453125,
   2187.975341796875,
   2186.132568359375,
   2184.376953125,
   2182.7041015625,
   2181.109619140625,
   2179.591064453125,
   2178.14599609375,
   2176.774169921875,
   2175.47412109375,
   2174.243408203125,
   2173.078369140625,
   2171.97509765625]],
 ['ASIANPAINT.NS',
  [3109.671142578125,
   3113.764404296875,
   3117.280517578125,
   3120.18896484375,
   3122.4951171875,
   3124.2685546875,
   3125.612060546875,
   3126.63330078125,
   3127.426513671875,
   3128.0673828125,
   3128.613525390625,
   3129.102294921875,
   3129.55712890625,
   3129.990966796875,
   3130.407958984375,
   3130.80

In [11]:


# Create a new list with the first column and differences, then sort it
sorted_list_15_days = sorted([(item[0], item[1][-1] - item[1][0]) for item in predictions_15], key=lambda x: x[1], reverse=True)



# Create a new list with the first column and differences, then sort it
sorted_list_30_days = sorted([(item[0], item[1][-1] - item[1][0]) for item in predictions_30], key=lambda x: x[1], reverse=True)



In [12]:
sorted_list_15_days

[('RELIANCE.NS', 38.120361328125),
 ('HDFCBANK.NS', 24.266357421875),
 ('ASIANPAINT.NS', 20.73681640625),
 ('KOTAKBANK.NS', 15.11474609375),
 ('UPL.NS', 10.91064453125),
 ('WIPRO.NS', 5.951141357421875),
 ('TATACHEM.NS', 4.70849609375),
 ('ADANIENT.NS', 4.544921875),
 ('ITC.NS', 1.44512939453125),
 ('ICICIBANK.NS', 0.1258544921875),
 ('BRFL.NS', 0.011285781860351562),
 ('SBIN.NS', -2.46600341796875),
 ('GAIL.NS', -4.019744873046875),
 ('POWERGRID.NS', -8.508346557617188),
 ('ONGC.NS', -8.939788818359375),
 ('JINDALSTEL.NS', -23.7669677734375),
 ('HINDALCO.NS', -24.211395263671875),
 ('TECHM.NS', -25.529052734375),
 ('COALINDIA.NS', -29.420745849609375),
 ('INFY.NS', -39.8807373046875),
 ('BHARTIARTL.NS', -49.271728515625),
 ('DLF.NS', -56.91925048828125),
 ('TCS.NS', -59.97802734375),
 ('BAJAJFINSV.NS', -63.6832275390625),
 ('GRASIM.NS', -70.169677734375),
 ('SUNPHARMA.NS', -71.698486328125),
 ('HCLTECH.NS', -77.900146484375),
 ('CIPLA.NS', -81.435791015625),
 ('TITAN.NS', -197.6879882

In [13]:
sorted_list_30_days

[('RELIANCE.NS', 58.71728515625),
 ('HDFCBANK.NS', 35.8695068359375),
 ('KOTAKBANK.NS', 27.050537109375),
 ('ASIANPAINT.NS', 24.784423828125),
 ('UPL.NS', 24.0760498046875),
 ('TATACHEM.NS', 13.65283203125),
 ('WIPRO.NS', 7.96697998046875),
 ('ITC.NS', 3.37384033203125),
 ('ICICIBANK.NS', 2.46514892578125),
 ('BRFL.NS', 0.023535966873168945),
 ('SBIN.NS', -1.8482666015625),
 ('GAIL.NS', -6.130683898925781),
 ('POWERGRID.NS', -12.10211181640625),
 ('ONGC.NS', -12.660324096679688),
 ('ADANIENT.NS', -19.848388671875),
 ('HINDALCO.NS', -35.5220947265625),
 ('COALINDIA.NS', -40.379974365234375),
 ('JINDALSTEL.NS', -41.6336669921875),
 ('TECHM.NS', -48.023193359375),
 ('INFY.NS', -64.656005859375),
 ('BHARTIARTL.NS', -71.1932373046875),
 ('DLF.NS', -77.24853515625),
 ('BAJAJFINSV.NS', -95.2314453125),
 ('SUNPHARMA.NS', -98.3240966796875),
 ('GRASIM.NS', -99.10400390625),
 ('TCS.NS', -100.34033203125),
 ('HCLTECH.NS', -105.27587890625),
 ('CIPLA.NS', -110.9979248046875),
 ('EICHERMOT.NS', -27

In [14]:
Close_result = [['ADANIENT.NS', []], ['ASIANPAINT.NS', [datetime.date(2023, 3, 2), datetime.date(2023, 3, 3), datetime.date(2023, 3, 6), datetime.date(2023, 3, 8), datetime.date(2023, 3, 9), datetime.date(2023, 3, 10), datetime.date(2023, 3, 13), datetime.date(2023, 3, 14)]], ['BAJAJFINSV.NS', [datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5), datetime.date(2023, 5, 8), datetime.date(2023, 5, 9), datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12), datetime.date(2023, 5, 15)]], ['BHARTIARTL.NS', []], ['BOSCHLTD.NS', [datetime.date(2023, 2, 14), datetime.date(2023, 2, 15), datetime.date(2023, 2, 16), datetime.date(2023, 2, 17), datetime.date(2023, 2, 20), datetime.date(2023, 2, 21), datetime.date(2023, 2, 22), datetime.date(2023, 2, 23), datetime.date(2023, 2, 24), datetime.date(2023, 2, 27), datetime.date(2023, 2, 28), datetime.date(2023, 3, 1), datetime.date(2023, 3, 2), datetime.date(2023, 3, 3), datetime.date(2023, 3, 6), datetime.date(2023, 3, 8), datetime.date(2023, 3, 9), datetime.date(2023, 3, 10), datetime.date(2023, 3, 13)]], ['BRFL.NS', []], ['CIPLA.NS', [datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26)]], ['COALINDIA.NS', []], ['DLF.NS', []], ['DRREDDY.NS', []], ['EICHERMOT.NS', [datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5), datetime.date(2023, 5, 8), datetime.date(2023, 5, 9), datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12)]], ['GAIL.NS', [datetime.date(2023, 2, 21), datetime.date(2023, 2, 22), datetime.date(2023, 2, 23), datetime.date(2023, 2, 24), datetime.date(2023, 2, 27), datetime.date(2023, 2, 28), datetime.date(2023, 3, 1), datetime.date(2023, 3, 2), datetime.date(2023, 3, 3), datetime.date(2023, 3, 6), datetime.date(2023, 3, 8), datetime.date(2023, 3, 9), datetime.date(2023, 3, 10), datetime.date(2023, 3, 13), datetime.date(2023, 3, 14), datetime.date(2023, 3, 15), datetime.date(2023, 3, 16), datetime.date(2023, 3, 17), datetime.date(2023, 3, 20)]], ['GRASIM.NS', [datetime.date(2023, 4, 5), datetime.date(2023, 4, 6), datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17)]], ['HDFCBANK.NS', [datetime.date(2023, 11, 7), datetime.date(2023, 11, 8), datetime.date(2023, 11, 9), datetime.date(2023, 11, 10), datetime.date(2023, 11, 13), datetime.date(2023, 11, 15), datetime.date(2023, 11, 16), datetime.date(2023, 11, 17), datetime.date(2023, 11, 20), datetime.date(2023, 11, 21), datetime.date(2023, 11, 22), datetime.date(2023, 11, 23), datetime.date(2023, 11, 24)]], ['HINDALCO.NS', [datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5)]], ['HCLTECH.NS', []], ['ICICIBANK.NS', [datetime.date(2023, 2, 27), datetime.date(2023, 2, 28), datetime.date(2023, 3, 1), datetime.date(2023, 3, 2), datetime.date(2023, 3, 3), datetime.date(2023, 3, 6), datetime.date(2023, 3, 8), datetime.date(2023, 3, 9), datetime.date(2023, 3, 10), datetime.date(2023, 3, 13), datetime.date(2023, 3, 14), datetime.date(2023, 3, 21), datetime.date(2023, 3, 22), datetime.date(2023, 3, 23), datetime.date(2023, 3, 24), datetime.date(2023, 3, 27), datetime.date(2023, 3, 28), datetime.date(2023, 3, 29), datetime.date(2023, 3, 31), datetime.date(2023, 4, 3), datetime.date(2023, 4, 5), datetime.date(2023, 4, 6), datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28)]], ['INFY.NS', [datetime.date(2023, 5, 23), datetime.date(2023, 5, 24), datetime.date(2023, 5, 25), datetime.date(2023, 5, 26), datetime.date(2023, 5, 29), datetime.date(2023, 5, 30), datetime.date(2023, 5, 31), datetime.date(2023, 6, 1), datetime.date(2023, 6, 2), datetime.date(2023, 6, 5), datetime.date(2023, 6, 6), datetime.date(2023, 6, 7), datetime.date(2023, 6, 8), datetime.date(2023, 6, 9), datetime.date(2023, 6, 12)]], ['ITC.NS', [datetime.date(2023, 2, 14), datetime.date(2023, 2, 15), datetime.date(2023, 2, 16), datetime.date(2023, 2, 17), datetime.date(2023, 2, 20), datetime.date(2023, 2, 21), datetime.date(2023, 2, 22), datetime.date(2023, 2, 23), datetime.date(2023, 2, 24), datetime.date(2023, 2, 27), datetime.date(2023, 2, 28), datetime.date(2023, 3, 1), datetime.date(2023, 3, 2), datetime.date(2023, 3, 3), datetime.date(2023, 3, 6)]], ['JINDALSTEL.NS', [datetime.date(2023, 6, 20), datetime.date(2023, 6, 21), datetime.date(2023, 6, 22), datetime.date(2023, 6, 23), datetime.date(2023, 6, 26), datetime.date(2023, 6, 27), datetime.date(2023, 6, 28), datetime.date(2023, 6, 30), datetime.date(2023, 7, 3), datetime.date(2023, 7, 4), datetime.date(2023, 7, 5), datetime.date(2023, 7, 6), datetime.date(2023, 7, 7)]], ['KOTAKBANK.NS', []], ['LT.NS', []], ['MARUTI.NS', []], ['NESTLEIND.NS', [datetime.date(2023, 3, 13), datetime.date(2023, 3, 14), datetime.date(2023, 3, 15), datetime.date(2023, 3, 16), datetime.date(2023, 3, 17), datetime.date(2023, 3, 20), datetime.date(2023, 3, 21), datetime.date(2023, 3, 22), datetime.date(2023, 3, 23), datetime.date(2023, 3, 24), datetime.date(2023, 3, 27), datetime.date(2023, 3, 28), datetime.date(2023, 3, 29), datetime.date(2023, 3, 31), datetime.date(2023, 4, 3), datetime.date(2023, 4, 5), datetime.date(2023, 4, 6), datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27)]], ['ONGC.NS', []], ['POWERGRID.NS', []], ['RELIANCE.NS', []], ['SBIN.NS', [datetime.date(2023, 3, 27), datetime.date(2023, 3, 28), datetime.date(2023, 3, 29), datetime.date(2023, 3, 31), datetime.date(2023, 4, 3), datetime.date(2023, 4, 5), datetime.date(2023, 4, 6), datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18), datetime.date(2023, 4, 19), datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5), datetime.date(2023, 5, 8), datetime.date(2023, 5, 9), datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12), datetime.date(2023, 5, 15)]], ['SUNPHARMA.NS', []], ['TATACHEM.NS', []], ['TCS.NS', [datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5), datetime.date(2023, 5, 8), datetime.date(2023, 5, 9), datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12), datetime.date(2023, 5, 15)]], ['TECHM.NS', []], ['TITAN.NS', [datetime.date(2023, 2, 21), datetime.date(2023, 2, 22), datetime.date(2023, 2, 23), datetime.date(2023, 2, 24)]], ['UPL.NS', [datetime.date(2023, 11, 22), datetime.date(2023, 11, 23), datetime.date(2023, 11, 24)]], ['WIPRO.NS', [datetime.date(2023, 4, 20), datetime.date(2023, 4, 21), datetime.date(2023, 4, 24), datetime.date(2023, 4, 25), datetime.date(2023, 4, 26), datetime.date(2023, 4, 27), datetime.date(2023, 4, 28), datetime.date(2023, 5, 2), datetime.date(2023, 5, 3), datetime.date(2023, 5, 4), datetime.date(2023, 5, 5), datetime.date(2023, 5, 8), datetime.date(2023, 5, 9), datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12), datetime.date(2023, 5, 15), datetime.date(2023, 5, 16), datetime.date(2023, 5, 17), datetime.date(2023, 5, 18), datetime.date(2023, 5, 19), datetime.date(2023, 5, 22)]]]

In [15]:
volume_result=[['ADANIENT.NS', []], ['ASIANPAINT.NS', []], ['BAJAJFINSV.NS', []], ['BHARTIARTL.NS', [datetime.date(2023, 6, 22), datetime.date(2023, 6, 23), datetime.date(2023, 6, 26), datetime.date(2023, 6, 27), datetime.date(2023, 6, 28), datetime.date(2023, 6, 30), datetime.date(2023, 7, 3), datetime.date(2023, 7, 4), datetime.date(2023, 7, 5), datetime.date(2023, 7, 6), datetime.date(2023, 7, 7), datetime.date(2023, 7, 10), datetime.date(2023, 7, 11), datetime.date(2023, 7, 12), datetime.date(2023, 7, 13)]], ['BOSCHLTD.NS', []], ['BRFL.NS', []], ['CIPLA.NS', [datetime.date(2023, 8, 31), datetime.date(2023, 9, 1), datetime.date(2023, 9, 4), datetime.date(2023, 9, 5), datetime.date(2023, 9, 6), datetime.date(2023, 9, 7), datetime.date(2023, 9, 8)]], ['COALINDIA.NS', []], ['DLF.NS', []], ['DRREDDY.NS', [datetime.date(2023, 6, 15), datetime.date(2023, 6, 16), datetime.date(2023, 6, 19), datetime.date(2023, 6, 20), datetime.date(2023, 6, 21), datetime.date(2023, 6, 22)]], ['EICHERMOT.NS', [datetime.date(2023, 8, 10), datetime.date(2023, 8, 11), datetime.date(2023, 8, 14), datetime.date(2023, 8, 16)]], ['GAIL.NS', []], ['GRASIM.NS', []], ['HDFCBANK.NS', [datetime.date(2023, 10, 25), datetime.date(2023, 10, 26), datetime.date(2023, 10, 27), datetime.date(2023, 10, 30), datetime.date(2023, 10, 31), datetime.date(2023, 11, 1)]], ['HINDALCO.NS', []], ['HCLTECH.NS', [datetime.date(2023, 4, 6), datetime.date(2023, 4, 10), datetime.date(2023, 4, 11), datetime.date(2023, 4, 12), datetime.date(2023, 4, 13), datetime.date(2023, 4, 17), datetime.date(2023, 4, 18)]], ['ICICIBANK.NS', []], ['INFY.NS', [datetime.date(2023, 5, 24), datetime.date(2023, 5, 25), datetime.date(2023, 5, 26), datetime.date(2023, 5, 29), datetime.date(2023, 5, 30), datetime.date(2023, 8, 28), datetime.date(2023, 8, 29), datetime.date(2023, 8, 30)]], ['ITC.NS', [datetime.date(2023, 3, 6), datetime.date(2023, 3, 8), datetime.date(2023, 3, 9), datetime.date(2023, 3, 10), datetime.date(2023, 3, 13), datetime.date(2023, 3, 14), datetime.date(2023, 3, 15), datetime.date(2023, 3, 16)]], ['JINDALSTEL.NS', []], ['KOTAKBANK.NS', [datetime.date(2023, 6, 20), datetime.date(2023, 6, 21), datetime.date(2023, 6, 22), datetime.date(2023, 6, 23), datetime.date(2023, 6, 26), datetime.date(2023, 6, 27), datetime.date(2023, 6, 28), datetime.date(2023, 6, 30), datetime.date(2023, 7, 3), datetime.date(2023, 7, 4), datetime.date(2023, 7, 5), datetime.date(2023, 7, 6), datetime.date(2023, 7, 7), datetime.date(2023, 7, 10), datetime.date(2023, 7, 11), datetime.date(2023, 7, 12), datetime.date(2023, 7, 13)]], ['LT.NS', []], ['MARUTI.NS', []], ['NESTLEIND.NS', []], ['ONGC.NS', [datetime.date(2023, 6, 21), datetime.date(2023, 6, 22), datetime.date(2023, 6, 23), datetime.date(2023, 6, 26), datetime.date(2023, 6, 27), datetime.date(2023, 6, 28), datetime.date(2023, 6, 30), datetime.date(2023, 7, 3), datetime.date(2023, 7, 4), datetime.date(2023, 7, 5), datetime.date(2023, 7, 6), datetime.date(2023, 7, 7), datetime.date(2023, 7, 10), datetime.date(2023, 7, 11), datetime.date(2023, 7, 12), datetime.date(2023, 7, 13)]], ['POWERGRID.NS', []], ['RELIANCE.NS', []], ['SBIN.NS', []], ['SUNPHARMA.NS', []], ['TATACHEM.NS', []], ['TCS.NS', []], ['TECHM.NS', []], ['TITAN.NS', []], ['UPL.NS', []], ['WIPRO.NS', [datetime.date(2023, 8, 24), datetime.date(2023, 8, 25), datetime.date(2023, 8, 28)]]]

In [16]:
volume_result_formatted = [
    [symbol, [date.strftime('%Y-%m-%d') for date in dates]] if dates else [symbol, []]
    for symbol, dates in volume_result
]

In [17]:
Close_result_formatted = [
    [symbol, [date.strftime('%Y-%m-%d') for date in dates]] if dates else [symbol, []]
    for symbol, dates in Close_result
]

In [18]:
Close_result_formatted

[['ADANIENT.NS', []],
 ['ASIANPAINT.NS',
  ['2023-03-02',
   '2023-03-03',
   '2023-03-06',
   '2023-03-08',
   '2023-03-09',
   '2023-03-10',
   '2023-03-13',
   '2023-03-14']],
 ['BAJAJFINSV.NS',
  ['2023-04-12',
   '2023-04-13',
   '2023-04-17',
   '2023-04-18',
   '2023-04-19',
   '2023-04-20',
   '2023-04-21',
   '2023-04-24',
   '2023-04-25',
   '2023-04-26',
   '2023-04-27',
   '2023-04-28',
   '2023-05-02',
   '2023-05-03',
   '2023-05-04',
   '2023-05-05',
   '2023-05-08',
   '2023-05-09',
   '2023-05-10',
   '2023-05-11',
   '2023-05-12',
   '2023-05-15']],
 ['BHARTIARTL.NS', []],
 ['BOSCHLTD.NS',
  ['2023-02-14',
   '2023-02-15',
   '2023-02-16',
   '2023-02-17',
   '2023-02-20',
   '2023-02-21',
   '2023-02-22',
   '2023-02-23',
   '2023-02-24',
   '2023-02-27',
   '2023-02-28',
   '2023-03-01',
   '2023-03-02',
   '2023-03-03',
   '2023-03-06',
   '2023-03-08',
   '2023-03-09',
   '2023-03-10',
   '2023-03-13']],
 ['BRFL.NS', []],
 ['CIPLA.NS',
  ['2023-04-17',
   '2023-04

In [19]:
volume_result_formatted

[['ADANIENT.NS', []],
 ['ASIANPAINT.NS', []],
 ['BAJAJFINSV.NS', []],
 ['BHARTIARTL.NS',
  ['2023-06-22',
   '2023-06-23',
   '2023-06-26',
   '2023-06-27',
   '2023-06-28',
   '2023-06-30',
   '2023-07-03',
   '2023-07-04',
   '2023-07-05',
   '2023-07-06',
   '2023-07-07',
   '2023-07-10',
   '2023-07-11',
   '2023-07-12',
   '2023-07-13']],
 ['BOSCHLTD.NS', []],
 ['BRFL.NS', []],
 ['CIPLA.NS',
  ['2023-08-31',
   '2023-09-01',
   '2023-09-04',
   '2023-09-05',
   '2023-09-06',
   '2023-09-07',
   '2023-09-08']],
 ['COALINDIA.NS', []],
 ['DLF.NS', []],
 ['DRREDDY.NS',
  ['2023-06-15',
   '2023-06-16',
   '2023-06-19',
   '2023-06-20',
   '2023-06-21',
   '2023-06-22']],
 ['EICHERMOT.NS', ['2023-08-10', '2023-08-11', '2023-08-14', '2023-08-16']],
 ['GAIL.NS', []],
 ['GRASIM.NS', []],
 ['HDFCBANK.NS',
  ['2023-10-25',
   '2023-10-26',
   '2023-10-27',
   '2023-10-30',
   '2023-10-31',
   '2023-11-01']],
 ['HINDALCO.NS', []],
 ['HCLTECH.NS',
  ['2023-04-06',
   '2023-04-10',
   '2023-04

In [29]:
from datetime import datetime
# Convert '2023-06-01' to a datetime object for comparison
cutoff_date = datetime.strptime('2023-06-01', '%Y-%m-%d')

# Check if manipulation dates are greater than '2023-06-01' for each company in the anomaly lists
manipulated_companies = set()
for anomaly_list in [Close_result_formatted, volume_result_formatted]:
    for company, manipulation_dates in anomaly_list:
        if any(datetime.strptime(date, '%Y-%m-%d') > cutoff_date for date in manipulation_dates):
            manipulated_companies.add(company)

# Remove companies from predictions list if in manipulated_companies
removed_companies = []
updated_predictions = []
for company, value in sorted_list_30_days:
    if company in manipulated_companies:
        removed_companies.append(company)
    else:
        updated_predictions.append((company, value))

# Print the updated predictions list
print("Updated Recommended Predictions:")
for prediction in updated_predictions:
    if(prediction[1]>0):
        print(prediction[0])
   
print("\nThe loss making companies:")
for prediction in updated_predictions:
    if(prediction[1]<0):
        print(prediction[0])
# Print the removed company names
print("\nRemoved Companies:")
for removed_company in removed_companies:
    print(removed_company)

Updated Recommended Predictions:
RELIANCE.NS
ASIANPAINT.NS
TATACHEM.NS
ITC.NS
ICICIBANK.NS
BRFL.NS

The loss making companies:
SBIN.NS
GAIL.NS
POWERGRID.NS
ADANIENT.NS
HINDALCO.NS
COALINDIA.NS
TECHM.NS
DLF.NS
BAJAJFINSV.NS
SUNPHARMA.NS
GRASIM.NS
TCS.NS
HCLTECH.NS
TITAN.NS
LT.NS
MARUTI.NS
NESTLEIND.NS
BOSCHLTD.NS

Removed Companies:
HDFCBANK.NS
KOTAKBANK.NS
UPL.NS
WIPRO.NS
ONGC.NS
JINDALSTEL.NS
INFY.NS
BHARTIARTL.NS
CIPLA.NS
EICHERMOT.NS
DRREDDY.NS
